In [5]:
from langchain.chat_models import ChatOpenAI

# 창의성 조절
chat = ChatOpenAI(
    temperature=0.1
)

In [6]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?"),
]

chat.predict_messages(messages)

SyntaxError: invalid syntax (3027070038.py, line 8)